Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# remove the exchange from the beginning of the symbol
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()
dow30['Symbol'] = dow30.apply(_symbol, axis=1)

dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Pharmaceutical industry
AAPL,Apple Inc.,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar Inc.,Construction and Mining
CVX,Chevron Corporation,Petroleum industry
CSCO,Cisco Systems,Information technology
KO,The Coca-Cola Company,Food industry


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and Mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services'}

In [7]:
# map dow30 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
dow30.index = dow30.index.str.replace('.', '-')
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AAPL,Apple Inc.,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar Inc.,US Stocks:Industrials
CVX,Chevron Corporation,US Stocks:Energy
CSCO,Cisco Systems,US Stocks:Technology
KO,The Coca-Cola Company,US Stocks:Consumer Staples


In [8]:
# make symbols list
symbols = list(dow30.index)
#symbols

In [9]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..............................


In [10]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,...,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT,DIS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,136.07,83.98,134.34,24.82,113.66,75.49,84.59,22.62,34.33,nan,...,43.89,74.46,59.24,91.02,91.33,35.10,63.29,64.08,73.57,87.38
2015-01-05,133.00,81.76,132.74,24.12,112.87,71.50,81.21,22.17,34.33,nan,...,43.18,74.10,58.17,89.92,89.82,34.81,61.89,62.82,73.36,86.10
2015-01-06,131.58,80.02,128.47,24.12,111.54,71.04,81.17,22.16,34.59,nan,...,42.93,73.76,57.20,89.12,89.64,35.16,61.49,62.98,73.92,85.64
2015-01-07,132.54,81.77,132.95,24.46,113.27,72.15,81.10,22.36,35.02,nan,...,43.81,74.15,56.93,90.64,90.56,34.93,62.31,64.59,75.88,86.52
2015-01-08,135.71,82.93,132.47,25.40,115.28,72.89,82.96,22.54,35.45,nan,...,44.83,75.00,58.59,92.52,94.88,35.68,63.15,65.39,77.48,87.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-28,203.04,160.13,237.94,124.61,247.02,241.08,103.79,52.90,55.29,68.42,...,136.46,134.85,238.10,159.70,411.92,56.49,227.30,52.66,142.03,178.65
2021-06-01,203.20,163.76,233.58,124.28,254.73,242.76,106.65,52.62,55.28,70.41,...,134.51,132.88,236.20,160.89,406.72,56.38,226.63,52.84,141.66,178.84
2021-06-02,203.29,164.12,235.16,125.06,255.62,243.46,108.08,52.96,55.50,68.95,...,134.17,133.46,234.62,160.20,405.85,56.65,229.66,53.83,141.35,177.00


In [11]:
# sample symbol
symbol = 'MMM'

In [12]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

31.041818758950246

In [13]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [14]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.16655976817076731

In [15]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.10171409149432402

In [16]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   134.61
2015-02-28   140.75
2015-03-31   137.66
2015-04-30   130.52
2015-05-31   133.60
2015-06-30   129.59
2015-07-31   127.10
2015-08-31   120.20
2015-09-30   119.89
2015-10-31   132.95
2015-11-30   133.29
2015-12-31   128.23
2016-01-31   128.54
2016-02-29   134.50
2016-03-31   142.87
2016-04-30   143.51
2016-05-31   145.28
2016-06-30   151.14
2016-07-31   153.94
2016-08-31   155.66
2016-09-30   153.05
2016-10-31   143.56
2016-11-30   150.11
2016-12-31   156.08
2017-01-31   152.80
2017-02-28   163.94
2017-03-31   168.32
2017-04-30   172.28
2017-05-31   180.95
2017-06-30   184.24
2017-07-31   178.03
2017-08-31   181.86
2017-09-30   186.83
2017-10-31   204.89
2017-11-30   217.50
2017-12-31   210.55
2018-01-31   224.09
2018-02-28   211.92
2018-03-31   197.53
2018-04-30   174.92
2018-05-31   178.69
2018-06-30   178.23
2018-07-31   192.36
2018-08-31   192.37
2018-09-30   192.18
2018-10-31   173.53
2018-11-30   190.93
2018-12-31   174.97
2019-01-31   183.93
2019-02-28   19

In [17]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2021-02-28    0.00
2021-03-31    0.10
2021-04-30    0.02
2021-05-31    0.04
2021-06-30    0.01
Freq: M, Name: MMM, Length: 78, dtype: float64

In [18]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.22058470745508066

In [19]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')